In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import pickle
import matplotlib.pyplot as plt

Run the below 2 Cells for using Image Data Generator- will take more time because it doesn't work with GPU.

In [ ]:
Dir = os.path.join(os.getcwd(), "dataset\\images")
train_dir = os.path.join(Dir, "train")
validation_dir = os.path.join(Dir, "validation")

In [ ]:
image_datagen = ImageDataGenerator(rescale=1/255)

train_generator = image_datagen.flow_from_directory(train_dir,
                                                    target_size=(48,48), 
                                                    batch_size=128,
                                                    class_mode='categorical',
                                                    color_mode='grayscale')

validation_generator = image_datagen.flow_from_directory(validation_dir,
                                                         target_size=(48,48),
                                                         batch_size=128,
                                                         class_mode='categorical',
                                                         color_mode='grayscale')

Run the below 2 cells to import data from the custom made pickle stored data- have to run "data_generator" file first(before running these 2 cells)

In [2]:
X, y, validation_X, validation_y = [], [], [], []

with open("X_train.pickle", "rb") as f:
    X = pickle.load(f)
    
with open("y_train.pickle", "rb") as f:
    y = pickle.load(f)
    
with open("X_test.pickle", "rb") as f:
    validation_X = pickle.load(f)
    
with open("y_test.pickle", "rb") as f:
    validation_y = pickle.load(f)


In [3]:
X = np.array(X, dtype=np.float32).reshape(-1, 48, 48, 1)
validation_X = np.array(validation_X, dtype=np.float32).reshape(-1, 48, 48, 1)

X /= 255
validation_X /= 255

y = tf.keras.utils.to_categorical(y, 7)
validation_y = tf.keras.utils.to_categorical(validation_y, 7)

In [4]:
nodes = 64

In [13]:
class ThresholdCallBack(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        if logs['val_accuracy'] >= 0.95:
            self.model.stop_training = True
        
thresholdcallback = ThresholdCallBack()

In [11]:
model = Sequential()

def define_model():
    model.add(Conv2D(nodes, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))
    
    model.add(Conv2D(2*nodes, kernel_size=(3,3), activation='relu'))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))
    
#     model.add(Conv2D(2*2*nodes, kernel_size=(3,3), activation='relu'))
#     model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
    
    model.add(Flatten())
    
    model.add(Dense(8*nodes, activation='relu'))
    model.add(Dropout(0.2))
    
    model.add(Dense(7, activation='softmax'))
    
    model.summary()
    
define_model()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 46, 46, 64)        640       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 23, 23, 64)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 23, 23, 64)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 21, 21, 128)       73856     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 10, 10, 128)       0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 10, 10, 128)       0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 12800)            

In [14]:
def train_model():
    model.compile(optimizer=tf.keras.optimizers.Adam(),
                  loss=tf.keras.losses.categorical_crossentropy,
                  metrics=['accuracy'])
        
    history = model.fit(X, y, 
                        epochs=300, 
                        batch_size = 32, 
                        shuffle=True, 
                        validation_data=(validation_X, validation_y),
                        callbacks = [thresholdcallback])
    
train_model()

Epoch 1/300
901/901 [==============================] - 15s 17ms/step - loss: 1.4763 - accuracy: 0.4316 - val_loss: 1.3828 - val_accuracy: 0.4758
Epoch 2/300
901/901 [==============================] - 15s 17ms/step - loss: 1.3921 - accuracy: 0.4631 - val_loss: 1.3202 - val_accuracy: 0.4918
Epoch 3/300
901/901 [==============================] - 15s 17ms/step - loss: 1.3248 - accuracy: 0.4912 - val_loss: 1.2858 - val_accuracy: 0.5132
Epoch 4/300
901/901 [==============================] - 15s 17ms/step - loss: 1.2675 - accuracy: 0.5163 - val_loss: 1.2641 - val_accuracy: 0.5225
Epoch 5/300
901/901 [==============================] - 15s 17ms/step - loss: 1.2113 - accuracy: 0.5406 - val_loss: 1.2307 - val_accuracy: 0.5368
Epoch 6/300
901/901 [==============================] - 16s 17ms/step - loss: 1.1692 - accuracy: 0.5548 - val_loss: 1.2358 - val_accuracy: 0.5405
Epoch 7/300
901/901 [==============================] - 15s 17ms/step - loss: 1.1206 - accuracy: 0.5769 - val_loss: 1.2001 - val_ac

901/901 [==============================] - 17s 19ms/step - loss: 0.3995 - accuracy: 0.8593 - val_loss: 1.5551 - val_accuracy: 0.5719
Epoch 58/300
901/901 [==============================] - 15s 17ms/step - loss: 0.3931 - accuracy: 0.8603 - val_loss: 1.5451 - val_accuracy: 0.5696
Epoch 59/300
901/901 [==============================] - 16s 17ms/step - loss: 0.3856 - accuracy: 0.8641 - val_loss: 1.5066 - val_accuracy: 0.5703
Epoch 60/300
901/901 [==============================] - 15s 17ms/step - loss: 0.3726 - accuracy: 0.8702 - val_loss: 1.5443 - val_accuracy: 0.5774
Epoch 61/300
901/901 [==============================] - 15s 17ms/step - loss: 0.3708 - accuracy: 0.8694 - val_loss: 1.5038 - val_accuracy: 0.5736
Epoch 62/300
901/901 [==============================] - 15s 17ms/step - loss: 0.3695 - accuracy: 0.8711 - val_loss: 1.6102 - val_accuracy: 0.5682
Epoch 63/300
901/901 [==============================] - 16s 17ms/step - loss: 0.3717 - accuracy: 0.8674 - val_loss: 1.5766 - val_accuracy

901/901 [==============================] - 15s 17ms/step - loss: 0.2667 - accuracy: 0.9093 - val_loss: 1.7208 - val_accuracy: 0.5730
Epoch 114/300
901/901 [==============================] - 15s 17ms/step - loss: 0.2540 - accuracy: 0.9112 - val_loss: 1.7352 - val_accuracy: 0.5783
Epoch 115/300
901/901 [==============================] - 15s 17ms/step - loss: 0.2551 - accuracy: 0.9129 - val_loss: 1.8332 - val_accuracy: 0.5784
Epoch 116/300
901/901 [==============================] - 15s 17ms/step - loss: 0.2524 - accuracy: 0.9139 - val_loss: 1.7629 - val_accuracy: 0.5720
Epoch 117/300
901/901 [==============================] - 15s 17ms/step - loss: 0.2630 - accuracy: 0.9117 - val_loss: 1.7860 - val_accuracy: 0.5736
Epoch 118/300
901/901 [==============================] - 15s 17ms/step - loss: 0.2493 - accuracy: 0.9154 - val_loss: 1.8084 - val_accuracy: 0.5716
Epoch 119/300
901/901 [==============================] - 15s 17ms/step - loss: 0.2423 - accuracy: 0.9158 - val_loss: 1.7774 - val_ac

901/901 [==============================] - 15s 17ms/step - loss: 0.2070 - accuracy: 0.9314 - val_loss: 1.8604 - val_accuracy: 0.5667
Epoch 169/300
901/901 [==============================] - 15s 17ms/step - loss: 0.2167 - accuracy: 0.9265 - val_loss: 1.7880 - val_accuracy: 0.5626s: 0
Epoch 170/300
901/901 [==============================] - 15s 17ms/step - loss: 0.2068 - accuracy: 0.9315 - val_loss: 1.9009 - val_accuracy: 0.5705
Epoch 171/300
901/901 [==============================] - 15s 17ms/step - loss: 0.2128 - accuracy: 0.9293 - val_loss: 1.8549 - val_accuracy: 0.5699
Epoch 172/300
901/901 [==============================] - 15s 17ms/step - loss: 0.2030 - accuracy: 0.9314 - val_loss: 1.9131 - val_accuracy: 0.5723
Epoch 173/300
901/901 [==============================] - 15s 17ms/step - loss: 0.2025 - accuracy: 0.9318 - val_loss: 1.8746 - val_accuracy: 0.5698
Epoch 174/300
901/901 [==============================] - 15s 17ms/step - loss: 0.2065 - accuracy: 0.9314 - val_loss: 1.8756 - va

901/901 [==============================] - 15s 17ms/step - loss: 0.1887 - accuracy: 0.9386 - val_loss: 1.9471 - val_accuracy: 0.5698
Epoch 223/300
901/901 [==============================] - 15s 17ms/step - loss: 0.1933 - accuracy: 0.9378 - val_loss: 2.0372 - val_accuracy: 0.5701
Epoch 224/300
901/901 [==============================] - 15s 17ms/step - loss: 0.1896 - accuracy: 0.9391 - val_loss: 1.9122 - val_accuracy: 0.5705
Epoch 225/300
901/901 [==============================] - 15s 17ms/step - loss: 0.1888 - accuracy: 0.9375 - val_loss: 1.9399 - val_accuracy: 0.5658
Epoch 226/300
901/901 [==============================] - 15s 17ms/step - loss: 0.1786 - accuracy: 0.9409 - val_loss: 1.9247 - val_accuracy: 0.5620
Epoch 227/300
901/901 [==============================] - 15s 17ms/step - loss: 0.1801 - accuracy: 0.9408 - val_loss: 1.9894 - val_accuracy: 0.5641
Epoch 228/300
901/901 [==============================] - 15s 17ms/step - loss: 0.1912 - accuracy: 0.9380 - val_loss: 1.9551 - val_ac

901/901 [==============================] - 16s 17ms/step - loss: 0.1720 - accuracy: 0.9440 - val_loss: 1.9943 - val_accuracy: 0.5692
Epoch 278/300
901/901 [==============================] - 15s 17ms/step - loss: 0.1733 - accuracy: 0.9426 - val_loss: 2.0104 - val_accuracy: 0.5650
Epoch 279/300
901/901 [==============================] - 15s 17ms/step - loss: 0.1703 - accuracy: 0.9453 - val_loss: 2.0524 - val_accuracy: 0.5654
Epoch 280/300
901/901 [==============================] - 15s 17ms/step - loss: 0.1686 - accuracy: 0.9446 - val_loss: 2.0026 - val_accuracy: 0.5675
Epoch 281/300
901/901 [==============================] - 15s 17ms/step - loss: 0.1749 - accuracy: 0.9440 - val_loss: 1.9985 - val_accuracy: 0.5652
Epoch 282/300
901/901 [==============================] - 15s 17ms/step - loss: 0.1716 - accuracy: 0.9439 - val_loss: 1.9627 - val_accuracy: 0.5665
Epoch 283/300
901/901 [==============================] - 15s 17ms/step - loss: 0.1767 - accuracy: 0.9428 - val_loss: 1.9739 - val_ac

In [15]:
model.save('saved_classifier.model')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: saved_classifier.model\assets
